In [84]:
import torch
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import numpy as np
import time
import gc
import pandas as pd

class ML():
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    transforms, dataloaders, dataset_sizes, class_names = None, None, None, None
    model, ciriterion, optimizer = None, None, None

    def __init__(self, transform, model, criterion, opt, lr):
        self.Empty()
        self.model = model.to(self.device)
        self.criterion = criterion
#         self.optimizer = opt(model.fc.parameters(), lr=lr)
        self.optimizer = opt(model.parameters(), lr=lr) #, weight_decay=0.0)
        self.optimizer.zero_grad() #초기화
        self.transforms = transform

    def Empty(self):
        torch.cuda.empty_cache()
        gc.collect()        
        
    def Data(self, train_sets, valid_sets, batch_size=4, shuffle=True, num_workers=2):
#         train_sets, valid_sets = datasets.ImageFolder(train_dir, self.transforms['train']), datasets.ImageFolder(valid_dir, self.transforms['valid'])
        self.dataloaders = { 'train':DataLoader(train_sets, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers), 
                             'valid':DataLoader(valid_sets, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers) }
        self.dataset_sizes = {'train':len(train_sets), 'valid':len(valid_sets)}
        self.class_names = train_sets.classes
        print("train: {}, valid: {}".format(self.dataset_sizes['train'], self.dataset_sizes['valid']))
        
    def Train(self, num_epochs=5, empty=True, name="Noname"):
        if empty: self.Empty()
        since = time.time()
        for epoch in range(num_epochs):
            print('[Epoch {}/{}]'.format(epoch+1, num_epochs))
            for phase in ['train', 'valid']:
                if phase == 'train': self.model.train()
                else: self.model.eval()

                running_loss, running_corrects = 0.0, 0
                for inputs, labels in self.dataloaders[phase]:
                    inputs, labels = inputs.to(self.device), labels.to(self.device)
                    outputs = self.model(inputs)
                    loss = self.criterion(outputs, labels)
                    if phase == 'train':
                        self.optimizer.zero_grad()
                        loss.backward()
                        self.optimizer.step()

                    _, preds = torch.max(outputs, 1)
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)

                epoch_loss, epoch_acc = running_loss / self.dataset_sizes[phase], running_corrects.double() / self.dataset_sizes[phase]
                print('{} loss: {:.4f}, acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
                
#     def Train2(self, num_epochs=5):
#         since = time.time()
#         for epoch in range(num_epochs):
#             print('[Epoch {}/{}]'.format(epoch+1, num_epochs))
#             for phase in ['train', 'valid']:
#                 if phase == 'train': self.model.train()
#                 else: self.model.eval()

#                 running_loss, running_corrects = 0.0, 0
#                 for inputs, labels in self.dataloaders[phase]:
#                     inputs, labels = inputs.to(self.device), labels.to(self.device)
#                     self.optimizer.zero_grad()
#                     with torch.set_grad_enabled(phase == 'train'):# 학습 시에만 연산 기록을 추적
#                         outputs = self.model(inputs)
#                         _, preds = torch.max(outputs, 1)
#                         loss = self.criterion(outputs, labels)
#                         if phase == 'train':
#                             loss.backward()
#                             self.optimizer.step()

#                     running_loss += loss.item() * inputs.size(0)
#                     running_corrects += torch.sum(preds == labels.data)

#                 epoch_loss, epoch_acc = running_loss / self.dataset_sizes[phase], running_corrects.double() / self.dataset_sizes[phase]
#                 print('{} loss: {:.4f}, acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
#         time_elapsed = time.time() - since
#         print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
                
    def Save(self, filename):
        torch.save(self.model.state_dict(), filename)
        print(filename, 'saved')
        
    def Load(self, filename, empty=True):
        if empty: self.Empty()
        self.model.load_state_dict(torch.load(filename))        
        self.model.eval()
        print(filename, 'loaded')
        
    def Valid(self, show_yn=False):
        running_corrects = 0
        for idx, (inputs, labels) in enumerate(self.dataloaders['valid']):
            outputs = self.model(inputs.to(self.device))
            probs = torch.nn.Softmax(dim=1)(outputs).cpu().detach().numpy()
            probs_max = np.max(probs, axis=1)
            preds_max = np.argmax(probs, axis=1)
            if show_yn and preds_max.all() != labels.numpy().all(): self.Show(inputs, preds_max, probs_max, labels)
            running_corrects += np.sum(preds_max == labels.numpy())
        print("acc {} valided".format(running_corrects / self.dataset_sizes['valid']))
        
    def Show(self, inputs, preds, vals, labels, figsize=(16,4)):
        plt.subplots(figsize=figsize)
        for idx, i in enumerate(inputs):
            plt.subplot(1, len(inputs), idx+1)
            plt.imshow(i.numpy().transpose((1, 2, 0)))
            plt.title("{} {} {:.2f}".format(preds[idx]==labels[idx], self.class_names[preds[idx]], vals[idx]))
            plt.axis('off')
        plt.show()
              
    def Predict(self, test_sets):
        test_loader = DataLoader(test_sets, shuffle=False)
        answers = []
        for inputs, labels in test_loader:
            outputs = self.model(inputs.to(self.device))
            probs = torch.nn.Softmax(dim=1)(outputs).cpu().detach().numpy()
            answers.append(probs)
#             answers.append(outputs.cpu().detach().numpy())
        return answers
        
    def Submit(self, test_dir, filename, show_yn=False):
        test_set = datasets.ImageFolder(test_dir, self.transforms['valid'])
        test_loader = DataLoader(test_set, shuffle=False)
        answers = []
        for inputs, labels in test_loader:
            outputs = self.model(inputs.to(self.device))
#             vals, preds = torch.max(outputs, 1)
            probs = torch.nn.Softmax(dim=1)(outputs).cpu().detach().numpy()
            probs_max = np.max(probs, axis=1)
            preds_max = np.argmax(probs, axis=1)            
            if show_yn: self.Show(inputs, preds_max, probs_max, labels)
            answers.extend(list(preds_max))

        submit_df = pd.DataFrame({'answer_value': answers})
        submit_df.to_csv(filename)
        print(submit_df.value_counts())
        print(filename, 'submited')        
        
#     def __str__(self):
#         return "dataloaders : {}".format(self.dataloaders)

In [85]:
# !pip install timm
import timm
# print(timm.list_models(pretrained=True))
# model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=7) #75.714
# model = timm.create_model('efficientnet_b4', pretrained=True, num_classes=7) #90
model = timm.create_model('resnet50', pretrained=True, num_classes=7) #best 95.714

In [86]:
transform = { 
    'train': transforms.Compose([ 
                        transforms.ToTensor(), 
#                         transforms.RandomVerticalFlip(),        
#                         transforms.RandomHorizontalFlip(),        
#                         transforms.Grayscale(num_output_channels=3),
#                         transforms.RandomGrayscale(),
#                         transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
#                         transforms.RandomResizedCrop(224),
#                         transforms.Resize((224,224)),
#                         transforms.CenterCrop(224),
#                         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
    ]),
    'valid': transforms.Compose([ 
                        transforms.ToTensor(),
#                         transforms.CenterCrop(224),
#                         transforms.Grayscale(num_output_channels=3),
#                         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
    ]) }

# model = models.resnet18(pretrained=True)
# model = models.resnet50(pretrained=True)
# model = models.resnet152(pretrained=True)
# for param in model.parameters():
#     param.requires_grad = False

# model.fc = torch.nn.Linear(model.fc.in_features, 7) #len(self.class_names))
# model.fc = torch.nn.Sequential(
#     torch.nn.Linear(model.fc.in_features, 14, bias=True),
#     torch.nn.BatchNorm1d(14),            
#     torch.nn.ReLU(inplace=True),
#     torch.nn.Linear(14, 7, bias=True))

# train_sets = datasets.ImageFolder('/kaggle/input/testdata/train/', transform['train'])
# valid_sets = datasets.ImageFolder('/kaggle/input/testdata2/train2/', transform['valid'])
# ml = ML(train_sets, valid_sets, transform, batch_size=7)
# ml.Network(model, torch.nn.CrossEntropyLoss(), torch.optim.Adam, lr=0.00005)
# ml.Train(num_epochs=20)
# ml.Save("test.ml")

In [87]:
# ml.Load("test.ml")
# ml.Valid()

In [88]:
# ml.Submit('/kaggle/input/testdata/test/', 'output.csv')#, show_yn=True)

In [89]:
#kfold 처리
from sklearn.model_selection import StratifiedKFold
from PIL import Image

k_fold_num = 5
batch_size = 7
num_epochs = 5

#kfold return 의 idx처리를 위해 DataFrame 용 커스텀데이터셋 정의
class MyDataset(Dataset):
    def __init__(self, df, transform=None, classes=None):
        self.df, self.transform, self.classes = df, transform, classes
    def __getitem__(self, idx):
        data = self.df.iloc[idx]
        image = Image.open(data['img_path'])
        if self.transform: image = self.transform(image)
        return image, data['label']
    def __len__(self): return len(self.df)    

sets = datasets.ImageFolder('/kaggle/input/testdata/train/', transform['train'])
kfold = StratifiedKFold(n_splits=k_fold_num)


df = pd.DataFrame(sets.imgs, columns=('img_path','label'))

# 클래스 별로 이미지 수가 다르기 때문에 imbalance 문제를 완화하기 위해 가장 많은 클래스 이미지 수 / 각 클래스 이미지 수로 나눈 값을 가중치로 사용.
# class_num = [329, 205, 235, 134, 151, 245, 399]
# class_weight = torch.tensor(np.max(class_num) / class_num).to(device='cuda:0', dtype=torch.float)

for i, (train_index, valid_index) in enumerate(kfold.split(sets.imgs, sets.targets)):
    
    print(f"\n** Fold_{i} **")
    train_sets = MyDataset(df.iloc[train_index], transform['train'], sets.classes)
    valid_sets = MyDataset(df.iloc[valid_index], transform['valid'], sets.classes)
    ml = ML(transform, model, torch.nn.CrossEntropyLoss(), torch.optim.Adam, lr=0.00005)
    ml.Data(train_sets, valid_sets, batch_size=batch_size)
    ml.Train(num_epochs=num_epochs, name=f"Fold_{i}")
    ml.Save("train_fold_{}.ml".format(i))


** Fold_0 **
train: 1358, valid: 340
[Epoch 1/5]
train loss: 1.8475, acc: 0.3100
valid loss: 1.7339, acc: 0.4382
[Epoch 2/5]
train loss: 1.5476, acc: 0.4389
valid loss: 1.3706, acc: 0.6147
[Epoch 3/5]
train loss: 1.1694, acc: 0.6303
valid loss: 0.9101, acc: 0.7882
[Epoch 4/5]
train loss: 0.8348, acc: 0.7879
valid loss: 0.5248, acc: 0.9029
[Epoch 5/5]
train loss: 0.5556, acc: 0.8697
valid loss: 0.2756, acc: 0.9441
Training complete in 1m 12s
train_fold_0.ml saved

** Fold_1 **
train: 1358, valid: 340
[Epoch 1/5]
train loss: 0.3979, acc: 0.9116
valid loss: 0.1170, acc: 0.9853
[Epoch 2/5]
train loss: 0.3399, acc: 0.9028
valid loss: 0.0802, acc: 0.9882
[Epoch 3/5]
train loss: 0.2570, acc: 0.9308
valid loss: 0.0657, acc: 0.9882
[Epoch 4/5]
train loss: 0.2118, acc: 0.9418
valid loss: 0.0583, acc: 0.9912
[Epoch 5/5]
train loss: 0.1990, acc: 0.9426
valid loss: 0.0768, acc: 0.9882
Training complete in 1m 12s
train_fold_1.ml saved

** Fold_2 **
train: 1358, valid: 340
[Epoch 1/5]
train loss: 0.

In [90]:
#ensemble 처리
# ml = ML(transform, model, torch.nn.CrossEntropyLoss(), torch.optim.Adam, lr=0.00005)
# ml.Data(train_sets, valid_sets, batch_size=batch_size)

test_sets = datasets.ImageFolder('/kaggle/input/testdata/test/', transform['valid'])
len_img, len_classes = len(test_sets.imgs), len(test_sets.classes)
rst = [[np.zeros(len_classes)] for i in range(len_img)]

for i in range(k_fold_num):
    ml.Load("train_fold_{}.ml".format(i))
#     ml.Valid(show_yn=True)
    rtn = ml.Predict(test_sets)
    for idx, r in enumerate(rtn):
        rst[idx] += r #probs sum
#         r = r.flatten()
#         rst[idx] += np.array([1 if max(r)==i else 0 for i in r]) #voting

train_fold_0.ml loaded
train_fold_1.ml loaded
train_fold_2.ml loaded
train_fold_3.ml loaded
train_fold_4.ml loaded


In [91]:
answers = [np.argmax(r/len_classes) for r in rst]
# answers = [np.argmax(r) for r in rst]
print(answers)
submission_df = pd.DataFrame({'answer_value': answers})
submission_df.to_csv('kflod.csv')
submission_df.value_counts()

#kfold best, 98.57

[2, 3, 3, 3, 3, 3, 4, 0, 3, 1, 6, 2, 0, 2, 3, 1, 2, 0, 6, 3, 3, 5, 2, 3, 0, 5, 1, 2, 0, 5, 4, 5, 6, 2, 0, 5, 5, 4, 2, 1, 4, 0, 2, 3, 1, 3, 0, 5, 5, 2, 6, 5, 4, 1, 5, 0, 4, 5, 1, 1, 5, 0, 6, 1, 1, 2, 4, 1, 1, 3, 2, 3, 0, 1, 1, 6, 2, 0, 3, 4, 1, 6, 1, 6, 6, 4, 3, 6, 1, 2, 2, 5, 1, 0, 5, 6, 3, 3, 1, 6, 5, 6, 6, 0, 3, 2, 5, 3, 0, 0, 4, 6, 2, 5, 4, 2, 0, 0, 5, 6, 4, 2, 2, 6, 4, 1, 5, 6, 0, 4, 1, 1, 6, 4, 4, 2, 5, 6, 5, 0, 4, 3, 1, 5, 0, 5, 4, 0, 2, 5, 6, 1, 6, 3, 2, 2, 0, 1, 4, 5, 2, 4, 6, 2, 3, 4, 1, 5, 6, 2, 1, 5, 3, 1, 0, 3, 2, 5, 3, 4, 2, 0, 3, 6, 0, 3, 3, 2, 0, 4, 4, 2, 2, 4, 4, 6, 6, 6, 3, 2, 5, 5, 4, 6, 2, 1, 3, 6, 0, 2, 3, 1, 1, 3, 1, 5, 2, 2, 0, 0, 0, 1, 2, 2, 2, 6, 3, 2, 5, 4, 3, 5, 0, 2, 4, 5, 0, 6, 4, 1, 6, 1, 2, 1, 6, 0, 5, 4, 6, 3, 4, 2, 5, 3, 1, 4, 2, 3, 4, 0, 6, 0, 2, 0, 1, 4, 1, 4, 0, 1, 1, 5, 6, 4, 6, 2, 4, 4, 3, 6, 6, 4, 0, 3, 3, 3, 0, 0, 5, 2, 5, 5, 2, 5, 2, 6, 5, 1, 5, 1, 5, 3, 0, 3, 3, 1, 6, 5, 0, 2, 0, 4, 3, 4, 4, 2, 5, 1, 2, 0, 6, 6, 1, 3, 3, 0, 0, 5, 2, 4, 6, 6, 6, 

answer_value
2               55
3               51
0               50
5               50
6               50
1               48
4               46
dtype: int64

In [92]:
# pseudo labeling
sets = datasets.ImageFolder('/kaggle/input/testdata/train/', transform['train'])
test_sets = datasets.ImageFolder('/kaggle/input/testdata/test/', transform['train'])

df_sets = pd.DataFrame(sets.imgs, columns=('img_path','label'))
df_test_sets = pd.DataFrame(test_sets.imgs, columns=('img_path','label'))
df_test_sets['label'] = answers
df_pseudo = pd.concat([df_sets, df_test_sets])
pseudo_sets = MyDataset(df_pseudo, transform['train'], sets.classes)

ml = ML(transform, model, torch.nn.CrossEntropyLoss(), torch.optim.Adam, lr=0.00005)
ml.Data(pseudo_sets, valid_sets, batch_size=batch_size)
ml.Train(num_epochs=num_epochs)
ml.Save("train_pseudo_labeling.ml")

train: 2048, valid: 339
[Epoch 1/5]
train loss: 0.0683, acc: 0.9800
valid loss: 0.0035, acc: 1.0000
[Epoch 2/5]
train loss: 0.0566, acc: 0.9829
valid loss: 0.0039, acc: 1.0000
[Epoch 3/5]
train loss: 0.0468, acc: 0.9907
valid loss: 0.0039, acc: 1.0000
[Epoch 4/5]
train loss: 0.0445, acc: 0.9888
valid loss: 0.0034, acc: 1.0000
[Epoch 5/5]
train loss: 0.0301, acc: 0.9907
valid loss: 0.0044, acc: 1.0000
Training complete in 1m 44s
train_pseudo_labeling.ml saved


In [93]:
ml.Load("train_pseudo_labeling.ml")

# ml.Valid(show_yn=True)
ml.Submit('/kaggle/input/testdata/test/', 'pseudo_labeling.csv')#, show_yn=True)

train_pseudo_labeling.ml loaded
answer_value
2               55
3               51
0               50
5               50
6               50
1               48
4               46
dtype: int64
pseudo_labeling.csv submited
